# Project Market Share

## Imports

In [12]:
import pandas as pd
from typing import NamedTuple

# Biogeme 
import biogeme.database as db
from biogeme.biogeme import BIOGEME
from biogeme.expressions import Expression
from biogeme.expressions import Variable
from biogeme.models import loglogit, logit, nested


# models
from models.logit_lmpc12_model0 import V_0, logprob_0, biogeme_0, results as res_mod0, chosen_alternative
from models.logit_lmpc12_model1 import V_1, logprob_1, biogeme_1, results as res_mod1
from models.logit_lmpc12_model2 import V_2, logprob_2, biogeme_2, results as res_mod2
from models.logit_lmpc12_model3 import V_3, logprob_3, biogeme_3, results as res_mod3
from models.logit_lmpc12_model4 import results_nested, lognested, biogeme_nested, nests



## Calculate Weights

In [2]:
# Define population sizes for each stratum
census = {
    'female_44_less': 2841376,
    'female_45_more': 1519948,
    'male_44_less': 2926408,
    'male_45_more': 1379198,
}

# Total population size
population_size = sum(census.values())

# Load the dataset
file_path = 'lpmc12.dat'
df = pd.read_csv('models/lpmc12.dat', sep='\t')

# Define filters for each stratum
filters = {
    'female_44_less': (df['female'] == 1) & (df['age'] <= 44),
    'female_45_more': (df['female'] == 1) & (df['age'] > 44),
    'male_44_less': (df['female'] == 0) & (df['age'] <= 44),
    'male_45_more': (df['female'] == 0) & (df['age'] > 44),
}

# Count the sample size in each stratum
sample_segments = {
    segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
}
print(f'Sample segments: {sample_segments}')

# Total sample size
total_sample = sum(sample_segments.values())
print(f'Sample size: {total_sample}')

# Calculate weights
normalizedWeight = {
    segment_name: census[segment_name] * total_sample / (segment_size * population_size)
    for segment_name, segment_size in sample_segments.items()
}
print(f'Weights: {normalizedWeight}')

# Insert weights into the dataset
for segment_name, segment_rows in filters.items():
    df.loc[segment_rows, 'weight'] = normalizedWeight[segment_name]

# Verify sum of weights
sum_weights = df['weight'].sum()
print(f'Sum of the weights: {sum_weights}')

normalizedWeight = Variable('weight')

Sample segments: {'female_44_less': np.int64(1631), 'female_45_more': np.int64(1034), 'male_44_less': np.int64(1451), 'male_45_more': np.int64(884)}
Sample size: 5000
Weights: {'female_44_less': np.float64(1.005031010413465), 'female_45_more': np.float64(0.8480333014252863), 'male_44_less': np.float64(1.1635155138048476), 'male_45_more': np.float64(0.9000757667545914)}
Sum of the weights: 5000.0


## Function

In [3]:
class IndicatorTuple(NamedTuple):
    """Tuple storing the value of an indicator, and the bounds on its confidence interval."""

    value: float
    lower: float
    upper: float

In [13]:
database = db.Database('lpmc12', df)


def market_share(utilities: dict[int, Expression], results) -> dict[str, IndicatorTuple]:
    """Calculate the market shares of all alternatives, given the
    specification of the utility functions.

    :param utilities: Specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.

    :return: A dictionary where each entry corresponds to an
        alternative, and associates its name with the IndicatorTuple
        containing the value of the market share, and the lower and
        upper bounds of the 90% confidence interval.
    """
    if results == results_nested:
        prob_walk = nested(utilities, None, nests, 1)
        prob_cycle = nested(utilities, None, nests, 2)
        prob_pt = nested(utilities, None, nests, 3)
        prob_car = nested(utilities, None, nests, 4)
    else:
        prob_walk = logit(utilities, None, 1)
        prob_cycle = logit(utilities, None, 2)
        prob_pt = logit(utilities, None, 3)
        prob_car = logit(utilities, None, 4)

    # Simulation setup
    simulate = {
        'weight': normalizedWeight,  # Assuming normalized weights are provided
        'Prob. walk': prob_walk,
        'Prob. cycle': prob_cycle,
        'Prob. PT': prob_pt,
        'Prob. car': prob_car,
    }
    
    # Creating Biogeme object
    biosim = BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results.get_beta_values())
    print(simulated_values[['Prob. walk', 'Prob. cycle', 'Prob. PT', 'Prob. car']].describe())
    
    # Confidence intervals
    if results == results_nested:
        betas = biogeme_nested.free_beta_names
    else:
        logprob = loglogit(utilities, None, chosen_alternative)
        biogeme = BIOGEME(database, logprob)
        betas = biogeme.free_beta_names
    sensitivity_betas = results.get_betas_for_sensitivity_analysis(
        betas, use_bootstrap=False
    )
    left, right = biosim.confidence_intervals(sensitivity_betas, 0.9)


    # Initialize market shares
    market_shares = {}

    # Iterate through alternatives
    for alt_name, prob_name in [
        ("Walking", "Prob. walk"),
        ("Cycling", "Prob. cycle"),
        ("Public transportation", "Prob. PT"),
        ("Car", "Prob. car"),
    ]:
        weighted_name = f"Weighted choice_prob. {alt_name.lower()}"

        # Calculate weighted probabilities
        simulated_values[weighted_name] = (
            simulated_values["weight"] * simulated_values[prob_name]
        )
        left[weighted_name] = left["weight"] * left[prob_name]
        right[weighted_name] = right["weight"] * right[prob_name]

        # Calculate mean values and bounds
        market_share_value = simulated_values[weighted_name].mean()
        market_share_lower = left[weighted_name].mean()
        market_share_upper = right[weighted_name].mean()

        # Store results in the market shares dictionary
        market_shares[alt_name] = IndicatorTuple(
            value=market_share_value,
            lower=market_share_lower,
            upper=market_share_upper,
        )

    return market_shares

## Calculate market shares

In [11]:
market_shares_base = market_share(V_0, res_mod0)
for alternative, indicator in market_shares_base.items():
    print(
        f'Market share for {alternative}: {100*indicator.value:.1f}% '
        f'[{100*indicator.lower:.1f}%, '
        f'{100*indicator.upper:.1f}%]'
    )

         Prob. walk  Prob. cycle      Prob. PT    Prob. car
count  5.000000e+03  5000.000000  5.000000e+03  5000.000000
mean   1.704001e-01     0.029200  3.574000e-01     0.443000
std    1.629612e-01     0.018439  1.821333e-01     0.220800
min    1.903461e-13     0.000043  9.526819e-07     0.002286
25%    8.750002e-03     0.019479  2.575017e-01     0.278642
50%    1.257326e-01     0.025677  3.302956e-01     0.401025
75%    3.145519e-01     0.033997  4.215278e-01     0.594204
max    6.333856e-01     0.250870  9.970501e-01     0.999367
Market share for Walking: 16.9% [15.9%, 17.9%]
Market share for Cycling: 2.9% [2.5%, 3.3%]
Market share for Public transportation: 35.8% [34.4%, 37.3%]
Market share for Car: 44.3% [42.8%, 46.1%]


In [14]:
market_shares_base = market_share(V_1, res_mod1)
for alternative, indicator in market_shares_base.items():
    print(
        f'Market share for {alternative}: {100*indicator.value:.1f}% '
        f'[{100*indicator.lower:.1f}%, '
        f'{100*indicator.upper:.1f}%]'
    )

         Prob. walk   Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5.000000e+03  5000.000000  5000.000000
mean   1.704008e-01  2.919996e-02     0.357400     0.443000
std    2.270605e-01  1.660377e-02     0.238669     0.213765
min    2.174021e-26  5.122602e-07     0.000627     0.000161
25%    8.570957e-05  1.832766e-02     0.185888     0.284840
50%    3.310869e-02  2.910390e-02     0.286574     0.464464
75%    3.151704e-01  3.868107e-02     0.441198     0.600420
max    8.677694e-01  1.640226e-01     0.999810     0.999295
Market share for Walking: 16.9% [15.9%, 17.8%]
Market share for Cycling: 2.9% [2.4%, 3.5%]
Market share for Public transportation: 36.0% [34.2%, 38.0%]
Market share for Car: 44.2% [42.2%, 46.2%]


In [15]:
market_shares_base = market_share(V_2, res_mod2)
for alternative, indicator in market_shares_base.items():
    print(
        f'Market share for {alternative}: {100*indicator.value:.1f}% '
        f'[{100*indicator.lower:.1f}%, '
        f'{100*indicator.upper:.1f}%]'
    )

         Prob. walk   Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5.000000e+03  5000.000000  5000.000000
mean   1.703993e-01  2.920025e-02     0.357392     0.443009
std    2.279146e-01  1.801238e-02     0.241310     0.221874
min    7.812762e-28  3.447750e-09     0.000791     0.000261
25%    9.163455e-05  1.758907e-02     0.180924     0.277342
50%    3.273473e-02  2.851475e-02     0.285892     0.460267
75%    3.104299e-01  3.900590e-02     0.448534     0.601886
max    8.739168e-01  1.529539e-01     0.999597     0.998956
Market share for Walking: 16.9% [15.8%, 18.2%]
Market share for Cycling: 2.9% [2.3%, 3.7%]
Market share for Public transportation: 36.0% [33.8%, 38.3%]
Market share for Car: 44.1% [41.6%, 46.5%]


In [16]:
market_shares_base = market_share(V_3, res_mod3)
for alternative, indicator in market_shares_base.items():
    print(
        f'Market share for {alternative}: {100*indicator.value:.1f}% '
        f'[{100*indicator.lower:.1f}%, '
        f'{100*indicator.upper:.1f}%]'
    )

         Prob. walk  Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5000.000000  5000.000000  5000.000000
mean   1.704009e-01     0.029201     0.357401     0.442998
std    2.292483e-01     0.017832     0.245960     0.218750
min    1.155219e-30     0.000046     0.007444     0.001268
25%    2.714156e-05     0.017530     0.166425     0.280431
50%    2.740173e-02     0.026994     0.284779     0.457531
75%    3.187701e-01     0.038115     0.474462     0.606570
max    8.955295e-01     0.157479     0.998519     0.990473
Market share for Walking: 16.9% [15.6%, 18.5%]
Market share for Cycling: 2.9% [2.3%, 3.7%]
Market share for Public transportation: 36.0% [33.8%, 38.4%]
Market share for Car: 44.1% [41.6%, 46.6%]


In [17]:
market_shares_base = market_share(V_3, results_nested)
for alternative, indicator in market_shares_base.items():
    print(
        f'Market share for {alternative}: {100*indicator.value:.1f}% '
        f'[{100*indicator.lower:.1f}%, '
        f'{100*indicator.upper:.1f}%]'
    )

         Prob. walk  Prob. cycle     Prob. PT    Prob. car
count  5.000000e+03  5000.000000  5000.000000  5000.000000
mean   1.704053e-01     0.029198     0.358718     0.441679
std    2.262141e-01     0.013285     0.254006     0.222438
min    4.843372e-28     0.001327     0.005325     0.002272
25%    5.039167e-05     0.020409     0.155179     0.273727
50%    3.071888e-02     0.028397     0.283240     0.448481
75%    3.192087e-01     0.036473     0.496332     0.607476
max    8.397208e-01     0.080191     0.995338     0.974470
Market share for Walking: 16.9% [15.3%, 18.7%]
Market share for Cycling: 2.9% [2.1%, 3.8%]
Market share for Public transportation: 36.2% [32.6%, 47.5%]
Market share for Car: 44.0% [33.5%, 48.0%]
